In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import mpl_lego as mplego
import pandas as pd

from hatespeech.keys import (annotator_race_to_col,
                             annotator_gender,
                             annotator_educ,
                             annotator_income,
                             annotator_religion_to_col,
                             annotator_sexual_orientation_to_col,
                             annotator_ideology)

%matplotlib inline

In [ ]:
mplego.style.use_latex_style()

In [ ]:
exp = "02"
base_path = os.path.join(os.environ['HOME'], 'projects/hatespeech-2021-fairml')
data_path = os.path.join(os.environ['HOME'], 'data/hatespeech/clean_qualtrics_irt_rollout.feather')
results_path = os.path.join(base_path, f'scaling/experiments/exp{exp}')

In [ ]:
data = pd.read_feather(data_path)

In [ ]:
black_target_results_path = os.path.join(results_path, f"exp{exp}_scores_black.2.txt")
white_target_results_path = os.path.join(results_path, f"exp{exp}_scores_white.2.txt")

In [ ]:
# Get severities for black-targeting comments
black_target_results = pd.read_csv(black_target_results_path, delimiter='\t', skiprows=1)
black_target_annotators = black_target_results['Judges']
black_target_severity = black_target_results['Measure']
# Get severities for white-targeting comments
white_target_results = pd.read_csv(white_target_results_path, delimiter='\t', skiprows=1)
white_target_annotators = white_target_results['Judges']
white_target_severity = white_target_results['Measure']

In [ ]:
print(f'Black-targeting: {black_target_severity.shape}')
print(f'White-targeting: {white_target_severity.shape}')

In [ ]:
# Get annotators that looked at both black- and white- targeting comments
common_annotators = black_target_annotators[black_target_annotators.isin(white_target_annotators)].values
black_target_common = black_target_results[black_target_annotators.isin(common_annotators)]
white_target_common = white_target_results[white_target_annotators.isin(common_annotators)]
print(f'Common annotators: {common_annotators.shape}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

bins = np.linspace(-6, 6, 51)
axes[0].hist(
    black_target_severity,
    bins=bins,
    density=True,
    linewidth=2,
    color='C0',
    label=r'\textbf{Black-Targeted}',
    histtype='step')
axes[0].hist(
    white_target_severity,
    bins=bins,
    density=True,
    linewidth=2,
    color='C1',
    label=r'\textbf{White-Targeted}',
    histtype='step')

axes[0].axvline(0, linestyle='--', color='grey')
axes[0].set_xlim([-5, 5])
axes[0].set_xlabel(r'\textbf{Severity} $(\alpha_b/\alpha_w)$', fontsize=20)
axes[0].set_ylabel(r'\textbf{Density}', fontsize=20)
axes[0].legend(loc=2, prop={'size': 13})
axes[0].tick_params(labelsize=15)

axes[1].hist(
    black_target_common['Measure'] - white_target_common['Measure'],
    bins=np.linspace(-5, 5, 21),
    histtype='step',
    linewidth=3,
    color='black')
axes[1].set_xlim([-5, 5])
axes[1].axvline(0, linestyle='--', color='grey')
axes[1].tick_params(labelsize=15)
axes[1].set_xlabel(r'\textbf{Severity Difference} $(\alpha_b - \alpha_w)$', fontsize=20)
axes[1].set_ylabel(r'\textbf{Frequency}', fontsize=20)

# plt.savefig('exp01_severities.pdf', bbox_inches='tight')

In [ ]:
# Black annotator severity difference
black_annotators = data[data[annotator_race_to_col['black']] == 1]['labeler_id'].values
b_annotator_b_target_sev = black_target_common[black_target_common['Judges'].isin(black_annotators)]['Measure'].values
b_annotator_w_target_sev = white_target_common[white_target_common['Judges'].isin(black_annotators)]['Measure'].values
b_annotator_diffs = b_annotator_b_target_sev - b_annotator_w_target_sev
# White annotator severity difference
white_annotators = data[data[annotator_race_to_col['white']] == 1]['labeler_id'].values
w_annotator_b_target_sev = black_target_common[black_target_common['Judges'].isin(white_annotators)]['Measure'].values
w_annotator_w_target_sev = white_target_common[white_target_common['Judges'].isin(white_annotators)]['Measure'].values
w_annotator_diffs = w_annotator_b_target_sev - w_annotator_w_target_sev

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

bins = np.linspace(-5, 5, 31)

ax.hist(
    b_annotator_diffs,
    bins=bins,
    histtype='step',
    linewidth=3,
    density=True,
    label=r'\textbf{Black Annotators}',
    color='C0')
ax.hist(
    w_annotator_diffs,
    bins=bins,
    histtype='step',
    linewidth=3,
    density=True,
    label=r'\textbf{White Annotators}',
    color='C1')
ax.set_xlim([-5, 5])
ax.axvline(0, linestyle='--', color='grey')
ax.tick_params(labelsize=15)
ax.set_xlabel(r'\textbf{Severity Difference} $(\alpha_b - \alpha_w)$', fontsize=20)
ax.set_ylabel(r'\textbf{Frequency}', fontsize=20)
ax.legend(loc='best', prop={'size': 14})

inset = fig.add_axes([0.23, 0.5, 0.15, 0.35])
inset.boxplot(
    x=[b_annotator_diffs, w_annotator_diffs],
    showfliers=False,
    widths=0.5,
    medianprops={'color': 'black', 'lw': 2},
    boxprops={'lw': 2},)
inset.set_xticklabels(['Black\nAnn.', 'White\nAnn.'])
inset.set_ylabel(r'$\alpha_b - \alpha_w$', fontsize=20)
inset.axhline(0, linestyle='--', color='gray')
inset.tick_params(labelsize=15)
# plt.savefig('exp01_white_vs_black_annotators.pdf', bbox_inches='tight')

# All Identity Groups

In [ ]:
def subset_annotator_severity(common, subset):
    return common[common['Judges'].isin(subset)]['Measure'].values

def calculate_sev_diffs(target1, target2, group):
    sev1 = subset_annotator_severity(target1, group)
    sev2 = subset_annotator_severity(target2, group)
    sev_diffs = sev1 - sev2
    return sev1, sev2, sev_diffs

In [ ]:
# Race
# Black annotators
black_annotators = data[data[annotator_race_to_col['black']] == 1]['labeler_id'].values
b_annotator_b_target_sev, b_annotator_w_target_sev, b_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, black_annotators)

# White annotators
white_annotators = data[data[annotator_race_to_col['white']] == 1]['labeler_id'].values
w_annotator_b_target_sev, w_annotator_w_target_sev, w_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, white_annotators)

In [ ]:
# Gender
# Male annotators
male_annotators = data[data['demo_gender'] == annotator_gender['male']]['labeler_id'].values
m_annotator_b_target_sev, m_annotator_w_target_sev, m_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, male_annotators)

# Female annotators 
female_annotators = data[data['demo_gender'] == annotator_gender['female']]['labeler_id'].values
f_annotator_b_target_sev, f_annotator_w_target_sev, f_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, female_annotators)

In [ ]:
# Education
# Some high school / High school grad
hs_annotators = data[
    (data['demo_educ'] == annotator_educ['some_high_school']) |
    (data['demo_educ'] == annotator_educ['high_school_grad'])
]['labeler_id'].values
hs_annotator_b_target_sev, hs_annotator_w_target_sev, hs_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, hs_annotators)
# Some college / College grad (AA) / College grad (BA)
col_annotators = data[
    (data['demo_educ'] == annotator_educ['some_college']) |
    (data['demo_educ'] == annotator_educ['college_grad_aa']) |
    (data['demo_educ'] == annotator_educ['college_grad_ba'])
]['labeler_id'].values
col_annotator_b_target_sev, col_annotator_w_target_sev, col_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, col_annotators)
# Master's / Professional Degree / PhD
grad_annotators = data[
    (data['demo_educ'] == annotator_educ['masters']) |
    (data['demo_educ'] == annotator_educ['professional_degree']) |
    (data['demo_educ'] == annotator_educ['phd'])
]['labeler_id'].values
grad_annotator_b_target_sev, grad_annotator_w_target_sev, grad_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, grad_annotators)

In [ ]:
# Income
# Low-income
li_annotators = data[
    (data['demo_income'] == annotator_income['<10k']) |
    (data['demo_income'] == annotator_income['10k-50k'])
]['labeler_id'].values
li_annotator_b_target_sev, li_annotator_w_target_sev, li_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, li_annotators)
# Middle-income
mi_annotators = data[
    (data['demo_income'] == annotator_income['50k-100k'])
]['labeler_id'].values
mi_annotator_b_target_sev, mi_annotator_w_target_sev, mi_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, mi_annotators)
# High-income
hi_annotators = data[
    (data['demo_income'] == annotator_income['100k-200k']) |
    (data['demo_income'] == annotator_income['>200k'])
]['labeler_id'].values
hi_annotator_b_target_sev, hi_annotator_w_target_sev, hi_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, hi_annotators)

In [ ]:
# Religion
# Christian annotators
christian_annotators = data[data[annotator_religion_to_col['christian']] == 1]['labeler_id'].values
ch_annotator_b_target_sev, ch_annotator_w_target_sev, ch_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, christian_annotators)
# Non-religious annotators
non_religious_annotators = data[
    (data[annotator_religion_to_col['atheist']] == 1) |
    (data[annotator_religion_to_col['nothing']] == 1)
]['labeler_id'].values
nr_annotator_b_target_sev, nr_annotator_w_target_sev, nr_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, non_religious_annotators)

In [ ]:
# Sexual Orientation
# Straight annotators
straight_annotators = data[data[annotator_sexual_orientation_to_col['straight']] == 1]['labeler_id'].values
st_annotator_b_target_sev, st_annotator_w_target_sev, st_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, straight_annotators)
# Non-straight annotators
other_annotators = data[
    (data[annotator_sexual_orientation_to_col['gay']] == 1) |
    (data[annotator_sexual_orientation_to_col['bisexual']] == 1) |
    (data[annotator_sexual_orientation_to_col['other']] == 1)
]['labeler_id'].values
oth_annotator_b_target_sev, oth_annotator_w_target_sev, oth_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, other_annotators)

In [ ]:
# Ideology
# Liberal annotators
liberal_annotators = data[
    (data['demo_ideology'] == annotator_ideology['extremely_liberal']) |
    (data['demo_ideology'] == annotator_ideology['liberal']) |
    (data['demo_ideology'] == annotator_ideology['slightly_liberal'])
]['labeler_id'].values
lib_annotator_b_target_sev, lib_annotator_w_target_sev, lib_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, liberal_annotators)
# Neutral annotators
neutral_annotators = data[data['demo_ideology'] == annotator_ideology['neutral']]['labeler_id'].values
neu_annotator_b_target_sev, neu_annotator_w_target_sev, neu_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, neutral_annotators)
# Conservative annotators
conservative_annotators = data[
    (data['demo_ideology'] == annotator_ideology['extremely_conservative']) |
    (data['demo_ideology'] == annotator_ideology['conservative']) |
    (data['demo_ideology'] == annotator_ideology['slightly_conservative'])
]['labeler_id'].values
con_annotator_b_target_sev, con_annotator_w_target_sev, con_annotator_diffs = \
    calculate_sev_diffs(black_target_common, white_target_common, conservative_annotators)

In [ ]:
md(
f"""|Demographic  | Group          | Severity Difference |
|------------- |---------------- |--------------------
|**Race**      | Black           | {np.median(b_annotator_diffs):0.3f} |
|              | White           | {np.median(w_annotator_diffs):0.3f} |
|**Gender**    | Men             | {np.median(m_annotator_diffs):0.3f} |
|              | Women           | {np.median(f_annotator_diffs):0.3f} |
|**Education** | High School     | {np.median(hs_annotator_diffs):0.3f}| 
|              | College         | {np.median(col_annotator_diffs):0.3f}|
|              | Graduate        | {np.median(grad_annotator_diffs):0.3f}|
|**Income**    | Low-income      | {np.median(li_annotator_diffs):0.3f}|
|              | Middle-income   | {np.median(mi_annotator_diffs):0.3f}|
|              | High-income     | {np.median(hi_annotator_diffs):0.3f}|
|**Religion**  | Christian       | {np.median(ch_annotator_diffs):0.3f}|
|              | Non-Religious   | {np.median(nr_annotator_diffs):0.3f}|
|**Sexual Orientation** | Straight | {np.median(st_annotator_diffs):0.3f}|
|                       | Not Straight | {np.median(oth_annotator_diffs):0.3f}|
|**Ideology**           | Liberal      | {np.median(lib_annotator_diffs):0.3f}|
|                       | Independent  | {np.median(neu_annotator_diffs):0.3f}|
|                       | Conservative | {np.median(con_annotator_diffs):0.3f}
""")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

width = 0.2
spacing = 0.18

# Race
ax.boxplot(
    x=[black_diffs, white_diffs],
    positions=[-spacing, spacing],
    showfliers=False,
    widths=width,
    medianprops={'color': 'black', 'lw': 1},
    boxprops={'lw': 1})
mplego.labels.add_significance_label(ax, bounds=[-spacing, spacing], which='top', label='***')
# Gender
ax.boxplot(
    x=[male_diffs, female_diffs],
    positions=[1 - spacing, 1 + spacing],
    showfliers=False,
    widths=width,
    medianprops={'color': 'black', 'lw': 1},
    boxprops={'lw': 1},)
mplego.labels.add_significance_label(ax, bounds=[1 - spacing, 1 + spacing], which='top', label='*')
# Education
ax.boxplot(
    x=[hs_diffs, college_diffs, grad_diffs],
    positions=[2 - 1.5 * spacing, 2, 2 + 1.5 * spacing],
    showfliers=False,
    widths=width,
    medianprops={'color': 'black', 'lw': 1},
    boxprops={'lw': 1},)
mplego.labels.add_significance_label(ax, bounds=[2 - 1.5 * spacing, 2 + 1.5 * spacing], which='top', label='*')
# Income
ax.boxplot(
    x=[li_diffs, mi_diffs, hi_diffs],
    positions=[3 - 1.5 * spacing, 3, 3 + 1.5 * spacing],
    showfliers=False,
    widths=width,
    medianprops={'color': 'black', 'lw': 1},
    boxprops={'lw': 1},)
# Religion
ax.boxplot(
    x=[christian_diffs, non_religious_diffs],
    positions=[4 - spacing, 4 + spacing],
    showfliers=False,
    widths=width,
    medianprops={'color': 'black', 'lw': 1},
    boxprops={'lw': 1},)
mplego.labels.add_significance_label(ax, bounds=[4 - spacing, 4 + spacing], which='top', label='***')

# Sexual Orientation
ax.boxplot(
    x=[straight_diffs, other_diffs],
    positions=[5 - spacing, 5 + spacing],
    showfliers=False,
    widths=width,
    medianprops={'color': 'black', 'lw': 1},
    boxprops={'lw': 1},)
mplego.labels.add_significance_label(ax, bounds=[5 - spacing, 5 + spacing], which='top', label='***')

# Ideology
ax.boxplot(
    x=[liberal_diffs, neutral_diffs, conservative_diffs],
    positions=[6 - 1.5 * spacing, 6, 6 + 1.5 * spacing],
    showfliers=False,
    widths=width,
    medianprops={'color': 'black', 'lw': 1},
    boxprops={'lw': 1},)
mplego.labels.add_significance_label(ax, bounds=[6 - 1.5 * spacing, 6 + 1.5 * spacing], which='top', label='***')

ax.axhline(0, linestyle='-', color='gray', lw=0.5)

ax.set_xticks([
    -spacing,
    spacing,
    1 - spacing,
    1 + spacing,
    2 - 1.5 * spacing,
    2.0001,
    2.045+ 1.5 * spacing,
    3 - 1.5 * spacing,
    3.0001,
    3 + 1.5 * spacing,
    4 - spacing,
    4 + spacing,
    5 - spacing,
    5 + spacing,
    6 - 1.5 * spacing,
    6.0001,
    6 + 1.5 * spacing
], minor=True)
ax.set_xticklabels(['B', 'W', 'M', 'W', 'H.S.', 'C.', 'Grad.', 'L', 'M', 'H', 'Ch.', 'Non-R', 'St.', 'Oth.', 'L', 'I', 'C'], minor=True)
ax.set_xticks([0, 1, 2, 3, 4, 5, 6])
ax.set_xticklabels(
    mplego.labels.bold_text(['Race', 'Gender', 'Education', 'Income', 'Religion', 'Sex. Ori.', 'Ideology']))
ax.tick_params(labelsize=13, which='major', pad=20)
ax.tick_params(labelsize=13, which='minor')
ax.set_xlim([-0.5, 6.4])
ax.set_ylim([-2.8, 2.8])
ax.set_ylabel(r'\textbf{Severity Difference}', fontsize=15)
plt.savefig('identity_groups_race.pdf', bbox_inches='tight')